# Homework 1

The assignment requires you to compute the TF-IDF measures for words in a collection of documents or corpus. TF-IDF stands for Term Frequency-Inverse Document Frequency and measures the relevance of a term <i><b>t</b></i> (t can be a word or a sequence of words called an n-gram) for a document in a collection. It is a combination of two measures as follows:

1. Term Frequency (TF) measures the frequency of the term t in a document d. If the term occurs n times in a document, then TF is usually the normalized frequency and is defined as:

	TF(t, d) = n/(number of terms in d)

If the TF measure is high, then the term occurs frequently in the document and is considered relevant for that document. 

2. Inverse Document Frequency (IDF) is

	IDF(t) = log(total number of documents/number of documents containing t)

IDF measures how important a term is in the collection. Terms such as "and" and "the" may occur across all documents and are not considered relevant. So, the IDF value for such terms will be low. 

Finally, TF-IDF(t, d) = TF(t,d) * IDF(t)

For an example, see the example section at http://www.tfidf.com  . 

Your corpus could be a single file where each line or paragraph is a document or it could be a directory 
with multiple files where each file is a document. See the attached hw1-input.txt for a trivial example of two documents. Each line is a document here and a newline character is the indication for the end of the document. We will use this example to illustrate the computation of TF-IDF. 

You may choose to represent the document collection as a Python list of strings in your program. See the code cell below for a representation of the two documents in hw1-input.txt. We will refer to this document collection as the <i><b>sample document collection</b></i> in this homework. 

IMPORTANT:

(1) You need to strip out ALL punctuation and convert words to lowercase in your program.

(2)<b> You cannot use any Python library that computes the TF-IDF scores. </b>
    
(3) <b>  You are required to use NumPy arrays and the functions/techniques discussed in class to find the denominators in the TF and IDF measures in Parts III and IV </b> (number of terms in document and number of documents containing t). If you use naive loops to count these measures, you will lose points. Both these measures require you to compute something across the rows/columns. Think of which aggregate functions you could you use in NumPy to do this easily. 

In [1]:
sample_document_collection = ['The car is driven on the road', 'The truck is driven on the highway']

In [2]:
print(sample_document_collection) 

## Part I  - 10 Points

Write a function called doc_vocab that takes a document corpus (could be a filename or a directory name with multiple files in it. You can assume the former for simplicity.) and returns a (1) vocabulary of words for the corpus and (2) the document collection (a list of documents). 

You can implement the vocabulary as a dictionary where the key is the word and the value (to which the key is mapped) is a unique index assigned to the word. 

The dictionary for the sample document collection could be something like 
{'the': 0, 'car': 1, 'is': 2, 'driven': 3, 'on': 4, 'road': 5, 'truck': 6, 'highway': 7}

Note that the actual indices you have could be different. They just have to be unique and consecutive i.e. for n words in the dictionary, the indices should range from 0 to n-1. Also note that words "the" and "The" are assumed to be the same.  


In [3]:
import re
import string
import numpy as np
from nltk.corpus import stopwords

In [4]:
# PART I: build lexicon and document collection
def doc_vocab(filename):
    collection = list(); lexicon = dict(); line_counter = 0; #limit=2
    with open(filename) as f:
        for line in f:
            line = line.translate(str.maketrans('', '', string.punctuation)).lower().strip()            
            collection.append(line)
            tokenized_sent = line.split()
            for word in tokenized_sent:
                if word not in lexicon:
                    lexicon[word] = len(lexicon)
    
            
    # you may import libraries and define additional helper functions outside this function
    # lexicon is a synonym for vocabulary
    
    return lexicon, collection

## Part II - 20 Points

Write a function called <i><b>doc_term_matrix</i></b> that takes a vocabulary and a document collection as parameters and returns a document term matrix. This matrix should be a two-dimensional NumPy array with the following shape: (number of documents in collection, number of terms in vocabulary). For the sample document collection, the array's shape will be (2, 8). If the array is M, then M[i, j] will contain the number of times term j occurs in document i. For the sample document collection, you may get something like 

[[2 1 1 1 1 1 0 0]

 [2 0 1 1 1 0 1 1]] . 

Note that the columns may be arranged differently based on the indices for the terms in the vocabulary. 

In [5]:
# You will need to initialize a NumPy array to store the document term matrix. 
# Which NumPy function will you use for this? What are the dimensions of this matrix?
# Fill this NumPy array with the appropriate values and return it as the value for this function
#You may use the collections package to count the number of times an element occurs in a list. 
#Note a use of collections below. You can modify it for use in Part II
import collections
dictfreq = collections.Counter("the key is in the car".split())

for key in dictfreq.keys():
    print(key+": "+str(dictfreq[key]))

the: 2
key: 1
is: 1
in: 1
car: 1


In [6]:
# PART II: document term matrix

#nrows: number of documents
#ncols: number of unique words
def doc_term_matrix(vocabulary, collection):
    # fill in your code here
    freq = np.zeros((len(collection), len(vocabulary)))
    for i in range(len(collection)):
        dictfreq = collections.Counter(collection[i].split()) #get frequency counts for all rows
        for key in dictfreq.keys():
            vocab_idx = vocabulary[key] #Get index of vocab word to know new coordinates in matrix
            freq[i, vocab_idx] = dictfreq[key]
    
    return freq

## Part III - 20 points

Write a function called <i><b>tf_matrix</i></b> that takes a document term matrix and returns a normalized frequency matrix that represents the TF scores for each term in each document. 


In [7]:
# PART III: normalized document term matrix
# Again, initialize a NumPy array with the appropriate dimensions and 
# then fill it with the appropriate values.
# You must use a NumPy aggregation function for computing the TF scores
# Think about making your code succinct.
def row_norm(row):    
    return row/np.nansum(row)

def tf_matrix(document_term_matrix):
    norm_freq = np.apply_along_axis(row_norm, axis=1, arr=document_term_matrix) 
    
    return norm_freq

## Part IV - 20 points

Write a function called <i><b>tf_idf_matrix</i></b> that takes a document term matrix and a normalized frequency matrix and returns a matrix that represents the TF-IDF scores for each term in the document. You can use np.log10 to compute the logarithm here. For the sample document collection, the function could return something like 

[[0.0 &nbsp;  0.04300429 &nbsp; 0.0 &nbsp; 0.0  &nbsp;  0.0 &nbsp;  0.04300429 &nbsp;   0.0 &nbsp;  0.0 &nbsp;]
  
 [0.0 &nbsp; 0.0 &nbsp;  0.0 &nbsp;  0.0 &nbsp;  0.0 &nbsp;  0.0 &nbsp; 0.04300429 &nbsp; 0.04300429 &nbsp;]]


In [8]:
# PART IV: TF_IDF matrix

def tf_idf_matrix(document_term_matrix, tf_matrix):
    idf_mat = np.log10(len(document_term_matrix)/np.count_nonzero(document_term_matrix, axis=0))
    return np.multiply(tf_matrix, idf_mat)

## Part V - 10 points

Write code to test your functions for the trivial test case: hw1-input.txt. 
You must print out 

(1) the vocabulary

(2) the document frequency matrix

(3) the normalized frequency matrix

(4) the tf-idf matrix 


Label each output with what it is. E.g. VOCABULARY, DOCUMENT FREQUENCY MATRIX, and so on.  

# (1) The vocabulary

In [9]:
vocab, collection = doc_vocab('hw1-input.txt')
vocab

{'the': 0,
 'car': 1,
 'is': 2,
 'driven': 3,
 'on': 4,
 'road': 5,
 'truck': 6,
 'highway': 7}

# (2) The document frequency matrix

In [10]:
doc_term_mat = doc_term_matrix(vocabulary=vocab, collection=collection)
doc_term_mat

array([[2., 1., 1., 1., 1., 1., 0., 0.],
       [2., 0., 1., 1., 1., 0., 1., 1.]])

# (3) The normalized frequency matrix

In [11]:
tf_mat = tf_matrix(doc_term_mat)
tf_mat

array([[0.28571429, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
        0.14285714, 0.        , 0.        ],
       [0.28571429, 0.        , 0.14285714, 0.14285714, 0.14285714,
        0.        , 0.14285714, 0.14285714]])

# (4) The tf-idf matrix

In [12]:
tf_idf = tf_idf_matrix(doc_term_mat, tf_mat)
tf_idf

array([[0.        , 0.04300429, 0.        , 0.        , 0.        ,
        0.04300429, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.04300429, 0.04300429]])

## Part VI - 20 points

Now test your code with a more complex input file(s). You may use <b><i>beatles_biography.txt</b></i> taken from https://www.notablebiographies.com/Ba-Be/Beatles.html. In this case, the documents could be defined by the newlines in the file. You are free to collapse the paragraphs to create fewer documents. You may also use other documents about the Beatles to do a broader analysis or you can choose a set of documents about a different topic. This part of the assignment is open-ended. 

Using the aggregate functions in NumPy, print out the maximum tf-idf scores for each document and their corresponding words. Note that it may be interesting to pick a few top words (words that have tf-idf scores close to the maximum). 

You can choose to (1) have a list of stop words to filter the documents and (2) have a list of words (say, 1-3 words i.e. n-grams) for each term instead of a single word. You may have to modify your function definitions in this case. Show your modified functions in this section. This part is optional and for extra credit only.

Explain your methods clearly and write your observations from your experiments.

### IMPORTANT: Your submission must be an Jupyter Notebook (.ipynb) file that is organized exactly like this notebook file. You can insert cells (or use the cells provided) after the instructions for each section in this file and then insert your code into the cells. To write your observations for Part VI, change the type of the cell to Markdown. 

In [13]:
b_vocab, b_collection = doc_vocab('beatles_biography.txt')
b_vocab

{'in': 0,
 'the': 1,
 '1960s': 2,
 'a': 3,
 'new': 4,
 'band': 5,
 'known': 6,
 'as': 7,
 'beatles': 8,
 'burst': 9,
 'on': 10,
 'pop': 11,
 'music': 12,
 'scene': 13,
 'and': 14,
 'changed': 15,
 'it': 16,
 'forever': 17,
 'members': 18,
 'included': 19,
 'george': 20,
 'harrison': 21,
 '1943–2001': 22,
 'john': 23,
 'lennon': 24,
 '1940–1980': 25,
 'paul': 26,
 'mccartney': 27,
 '1942–': 28,
 'ringo': 29,
 'starr': 30,
 '1940–': 31,
 'with': 32,
 'release': 33,
 'of': 34,
 'three': 35,
 'anthologies': 36,
 'collections': 37,
 'mid1990s': 38,
 'remain': 39,
 'one': 40,
 'bestselling': 41,
 'musical': 42,
 'groups': 43,
 'all': 44,
 'time': 45,
 'came': 46,
 'from': 47,
 'liverpool': 48,
 'england': 49,
 'were': 50,
 'originally': 51,
 'inspired': 52,
 'by': 53,
 'simple': 54,
 'guitarandwashboard': 55,
 'style': 56,
 'skiffle': 57,
 'was': 58,
 'lively': 59,
 'type': 60,
 'acoustic': 61,
 'nonelectric': 62,
 'that': 63,
 'used': 64,
 'songs': 65,
 'british': 66,
 'american': 67,
 'fol

In [14]:
len(b_vocab)

712

In [15]:
b_doc_term_mat = doc_term_matrix(vocabulary=b_vocab, collection=b_collection)
b_doc_term_mat

array([[2., 7., 1., ..., 0., 0., 0.],
       [1., 3., 0., ..., 0., 0., 0.],
       [7., 9., 0., ..., 0., 0., 0.],
       ...,
       [3., 5., 0., ..., 0., 0., 0.],
       [2., 4., 0., ..., 0., 0., 0.],
       [4., 4., 0., ..., 1., 1., 1.]])

In [16]:
b_tf_mat = tf_matrix(b_doc_term_mat)
b_tf_mat

array([[0.03448276, 0.12068966, 0.01724138, ..., 0.        , 0.        ,
        0.        ],
       [0.01492537, 0.04477612, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07      , 0.09      , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.04225352, 0.07042254, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03225806, 0.06451613, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05882353, 0.05882353, 0.        , ..., 0.01470588, 0.01470588,
        0.01470588]])

In [17]:
b_tf_idf = tf_idf_matrix(b_doc_term_mat, b_tf_mat)
b_tf_idf

array([[0.00166568, 0.        , 0.02204748, ..., 0.        , 0.        ,
        0.        ],
       [0.00072097, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00338133, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00204104, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00155822, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00284145, 0.        , 0.        , ..., 0.0188052 , 0.0188052 ,
        0.0188052 ]])

In [18]:
# PART I: build lexicon and document collection
def doc_vocab_v2(filename):
    
    collection = list(); lexicon = dict()
    stop_words = set(stopwords.words('english'))
    with open(filename) as f:
        for line in f:
            line = line.translate(str.maketrans('', '', string.punctuation)).lower().strip() #remove punctuation           
            re_sw = re.compile(r"\b(" + "|".join(stop_words) + ")\\W") #regex to remove stopwords
            line = re_sw.sub("", line) #remove stopwords
            collection.append(line)
            tokenized_sent = line.split()
            
            
            for word in tokenized_sent:
                if word not in lexicon and word not in stop_words:
                    lexicon[word] = len(lexicon)
    
            
    # you may import libraries and define additional helper functions outside this function
    # lexicon is a synonym for vocabulary
    
    return lexicon, collection

In [19]:
b_vocab_v2, b_collection_v2 = doc_vocab_v2('beatles_biography.txt')
b_vocab_v2

{'1960s': 0,
 'new': 1,
 'band': 2,
 'known': 3,
 'beatles': 4,
 'burst': 5,
 'pop': 6,
 'music': 7,
 'scene': 8,
 'changed': 9,
 'forever': 10,
 'members': 11,
 'included': 12,
 'george': 13,
 'harrison': 14,
 '1943–2001': 15,
 'john': 16,
 'lennon': 17,
 '1940–1980': 18,
 'paul': 19,
 'mccartney': 20,
 '1942–': 21,
 'ringo': 22,
 'starr': 23,
 '1940–': 24,
 'release': 25,
 'three': 26,
 'anthologies': 27,
 'collections': 28,
 'mid1990s': 29,
 'remain': 30,
 'one': 31,
 'bestselling': 32,
 'musical': 33,
 'groups': 34,
 'time': 35,
 'came': 36,
 'liverpool': 37,
 'england': 38,
 'originally': 39,
 'inspired': 40,
 'simple': 41,
 'guitarandwashboard': 42,
 'style': 43,
 'skiffle': 44,
 'lively': 45,
 'type': 46,
 'acoustic': 47,
 'nonelectric': 48,
 'used': 49,
 'songs': 50,
 'british': 51,
 'american': 52,
 'folk': 53,
 'popular': 54,
 'later': 55,
 'us': 56,
 'artists': 57,
 'elvis': 58,
 'presley': 59,
 '1935–1977': 60,
 'buddy': 61,
 'holly': 62,
 '1936–1959': 63,
 'little': 64,
 '

In [20]:
b_collection_v2

['1960s new band known beatles burst pop music scene changed forever band members included george harrison 1943–2001 john lennon 1940–1980 paul mccartney 1942– ringo starr 1940– release three anthologies collections mid1990s beatles remain one bestselling musical groups time',
 'beatles came liverpool england originally inspired simple guitarandwashboard style skiffle music skiffle lively type acoustic nonelectric music used songs british american folk popular music later us pop artists elvis presley 1935–1977 buddy holly 1936–1959 little richard 1932– influenced four members beatles early interest music',
 'beatles started john lennon formed group called quarrymen 1956 paul mccartney joined group guitarist 1957 fourteenyear old george harrison though skilled guitarist initially impress seventeenyearold lennon eventually permanent spot developing group beatles went several additional members well several name changes quarrymen became johnny moondogs later called silver beatles eventual

In [21]:
len(b_collection_v2)

19

# As seen below, we have 82 stop_words in this document (712 before removing stop words and 630 after)

In [22]:
len(b_vocab_v2)

629

In [23]:
b_doc_term_mat_v2 = doc_term_matrix(vocabulary=b_vocab_v2, collection=b_collection_v2)
b_doc_term_mat_v2

array([[1., 1., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 3., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [24]:
b_tf_mat_v2 = tf_matrix(b_doc_term_mat_v2)
b_tf_mat_v2

array([[0.02631579, 0.02631579, 0.05263158, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.02083333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06976744, 0.02325581, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.025     , 0.025     ,
        0.025     ]])

In [25]:
b_tf_idf_v2 = tf_idf_matrix(b_doc_term_mat_v2, b_tf_mat_v2)
b_tf_idf_v2

array([[0.03365141, 0.01317375, 0.02634749, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.01042922, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03492575, 0.01164192, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.03196884, 0.03196884,
        0.03196884]])

# The functions below are used to extract top 3 words for each document

In [26]:
def top3(row):
    """
    Use argsort() to return a list of indexes of array in sorted order.
    Pull the last 3 into a list(corresponding to the entries with the 3 highest tf-idf scores)
    """
    return row.argsort()[-3:] 

def get_word(idx):
    """
    Search for key associated to given numeric value. if none found, return empty string
    """
    for k,v in b_vocab_v2.items():
        if v==idx:
            return k
    return ""
def get_words(row):
    """
    Returns a list of words using get_word()
    """
    return [get_word(i) for i in row]


def top_words_by_doc():
    top_word_idx = np.apply_along_axis(top3, axis=1, arr=b_tf_idf_v2) #Get top 3 words by tf_idf score for each doc
    top_words = np.apply_along_axis(get_words, axis=1, arr=top_word_idx)
    return top_words
    

top_words = top_words_by_doc()

In [27]:
top_words

array([['bestselling', 'scene', '1960s'],
       ['buddy', 'music', 'skiffle'],
       ['eventually', 'guitarist', 'quarrymen'],
       ['named', 'drummer', 'manager'],
       ['right', '1962', 'martin'],
       ['remained', 'show', 'please'],
       ['weeks', 'week', 'top'],
       ['days', 'help', 'critics'],
       ['albums', 'classical', 'soul'],
       ['distanced', 'misundersto', 'said'],
       ['acclaimed', 'critics', 'india'],
       ['compilation', 'bbc', 'fields'],
       ['guitar', 'like', 'growing'],
       ['animated', 'money', 'submarine'],
       ['remainder', '1933–', 'ono'],
       ['album', 'best', 'together'],
       ['end', '1970', '1971'],
       ['city', 'new', 'york'],
       ['rock', 'roll', 'inducted']], dtype='<U11')

In [28]:
len(top_words)

19

Based on the results of the analysis, we have 19 documents<br>
"critics" is the only complete word that appears in more than one document<br>
However, if we had performed lemmatization, we would see that other root forms like "remain", "week", "album", and 
"best" also appear two or more times.
